# MCMC Decryption Metropolis algorithm

In [ ]:
import string
import numpy as np
import re
from random import randrange
from random import random

spaces = re.compile('([^A-Z]|\s+|\n)')

In [9]:
ind = dict()
txt = dict()

# Alphabet
allowed = string.ascii_uppercase + " "

# Make lookup tables for the markov chain
for i, c in enumerate(allowed):
    ind[c] = i
    txt[i] = c
    
    
# Read in reference text
transitions = np.ones((len(allowed), len(allowed)))
last = 'A'
with open('./warandpeace.txt') as f:
    for x in f:
        for c in re.sub(spaces, ' ', x.upper()):
            if c in allowed:
                transitions[ind[c], ind[last]] += 1
                last = c

In [4]:
# make normalized markov chain
# Useful for generating text
M = np.copy(transitions)
for i in range(len(allowed)):
    M[:,i] = M[:,i] / np.sum(M[:,i])

In [10]:
#Encode/Decode methods
def decode(text, key, alphabet):
    return text.translate(str.maketrans(key, alphabet))

def encode(text, key, alphabet):
    return text.translate(str.maketrans(alphabet, key))

# Return transposed key
def transpose(key):
    i, j = 0, 0
    while i == j:
        i, j = randrange(len(key)), randrange(len(key))
    keyl = list(key)
    keyl[i], keyl[j] = keyl[j], keyl[i]
    return ''.join(keyl)

# Score function (log)
def log_pi(inpstr, matrix):
    total = 0
    ins = re.sub(spaces, ' ', inpstr)
    for i in range(0, len(inpstr)-1):
        total += np.log(matrix[ind[ins[i]], ind[ins[i+1]]])
    return total

In [76]:
# generate random key
e_key = ''.join(np.random.permutation(list(allowed)))
print(e_key)
# d_key = decode(allowed, e_key, allowed)
# print(d_key)

# read in plain text message
plaintext = ""
with open('./example.txt') as f:
    for line in f:
        plaintext += line.upper()
print(plaintext[:200])

# generate cipher text from message
ciphertext = encode(plaintext, e_key, allowed)
print(ciphertext[:200])

print(decode(ciphertext, e_key, allowed)[:200])
print(log_pi(decode(ciphertext, e_key, allowed), transitions))

SWIUKENHDTMPLAXQJYZO RGFVBC
IT WAS IMPOSSIBLE TO GIVE BATTLE BEFORE INFORMATION HAD BEEN COLLECTED,
THE WOUNDED GATHERED IN, THE SUPPLIES OF AMMUNITION REPLENISHED, THE
SLAIN RECKONED UP, NEW OFFICERS APPOINTED TO REPLACE THOSE 
DOCGSZCDLQXZZDWPKCOXCNDRKCWSOOPKCWKEXYKCDAEXYLSODXACHSUCWKKACIXPPKIOKU,
OHKCGX AUKUCNSOHKYKUCDA,COHKCZ QQPDKZCXECSLL ADODXACYKQPKADZHKU,COHK
ZPSDACYKIMXAKUC Q,CAKGCXEEDIKYZCSQQXDAOKUCOXCYKQPSIKCOHXZKC
IT WAS IMPOSSIBLE TO GIVE BATTLE BEFORE INFORMATION HAD BEEN COLLECTED,
THE WOUNDED GATHERED IN, THE SUPPLIES OF AMMUNITION REPLENISHED, THE
SLAIN RECKONED UP, NEW OFFICERS APPOINTED TO REPLACE THOSE 
17836.370079


# Metropolis Algorithm

In [83]:
# Initial key 
maxkey = allowed
maxscore = log_pi(decode(ciphertext, maxkey, allowed), transitions)
# Number of restarts
for j in range(2):
    iterkey = ''.join(np.random.permutation(list(allowed)))
    oscore = log_pi(decode(ciphertext, iterkey, allowed), transitions)
    # Number of iterations
    for i in range(7000):
        nkey = transpose(iterkey)
        nscore = log_pi(decode(ciphertext, nkey, allowed), transitions)
        if nscore > maxscore:
            print(nscore)
            maxscore = nscore
            maxkey = nkey
        if (nscore/oscore)**25 > random():
            iterkey = nkey
            oscore = nscore
        
print(log_pi(decode(ciphertext, maxkey, allowed), transitions))
print(decode(ciphertext, maxkey, allowed)[:200])
print(maxkey, e_key)

12110.1869312
12795.5533113
12922.9137601
13242.3531619
13286.4631218
13326.9694743
13544.2728206
14077.5959682
14236.2741154
14558.609908
14563.7930253
15239.89489
15586.945874
15744.888755
15887.9595059
15957.5257969
15978.4934199
16269.1675993
16455.0757201
16464.7812161
16502.3716797
16518.7362495
16549.9081172
16592.6323476
16686.1533627
16760.2598394
16760.2598394
RS PTA RNFEAARBHO SE IRQO BTSSHO BOZEWO RDZEWNTSRED MTG BOOD YEHHOYSOG,
SMO PELDGOG ITSMOWOG RD, SMO ALFFHROA EZ TNNLDRSRED WOFHODRAMOG, SMO
AHTRD WOYCEDOG LF, DOP EZZRYOWA TFFERDSOG SE WOFHTYO SMEAO 
ZWMAXQUPNBF HLKGRDOSVTYJIEC SWIUKENHDTMPLAXQJYZO RGFVBC


# Examples

## Encrypting / Decrypting

In [50]:
text = "EXAMPLE PLAIN TEXT TO ENCRYPT"
ekey = "XEBPROHYAUFTIDSJLKZMWVNGQC"
alph = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"

cipher = encode(text, ekey, alph)

print("Text:    ", text)
print("E Key:   ", ekey)
print("Cipher:  ", cipher)
print("D Key:   ", decode(ekey, alph, ekey))
print("Decoded: ", decode(cipher, ekey, alph))

Text:     EXAMPLE PLAIN TEXT TO ENCRYPT
E Key:    XEBPROHYAUFTIDSJLKZMWVNGQC
Cipher:   RGXIJTR JTXAD MRGM MS RDBKQJM
D Key:    GREJKSYQXWOMAPZUTFCINVDHLB
Decoded:  EXAMPLE PLAIN TEXT TO ENCRYPT


## Generating Text

In [80]:
sample = ""
last = 'A'
for i in range(50):
    x = np.random.choice(list(allowed), p=M[:,ind[last]])
    last = x
    sample += x
sample

'F YINDO TOTE THR RUER TOUTHED THE  OPE  WERONEX WE'